# Multiple chains

In [27]:
!pip install --upgrade --quiet langchain langchain-openai

In [2]:
import os
os.environ["OPENAI_API_KEY"]      = "sk-*******************************************************"

In [28]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"person": "obama", "language": "korean"})

'시카고는 일리노이 주에 위치한 도시입니다.'

In [29]:
from langchain_core.runnables import RunnablePassthrough

prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else:"
)
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)
prompt4 = ChatPromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?"
)

model_parser = model | StrOutputParser()

color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser
question_generator = (
    color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4
)

prompt = question_generator.invoke("warm")
model.invoke(prompt)

AIMessage(content='The color of orange is orange, and the flag of Cyprus is white with a copper-colored map of the island in the center.', response_metadata={'finish_reason': 'stop', 'logprobs': None})

## Branching and Merging

In [30]:
planner = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}")
    | ChatOpenAI()
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "List the pros or positive aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)
arguments_against = (
    ChatPromptTemplate.from_template(
        "List the cons or negative aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Generate a final response given the critique"),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

chain.invoke({"input": "scrum"})

'While Scrum offers numerous benefits such as promoting collaboration, adaptability, and continuous improvement, there are indeed some potential drawbacks to consider. These include the need for a high level of commitment from team members, challenges with large or complex projects, potential undefined roles and responsibilities, scope creep risks, and dependencies on strong team dynamics.\n\nTo address these concerns and maximize the effectiveness of Scrum, it is crucial to ensure clear communication, well-defined roles, and a strong commitment from all team members. Additionally, proper project planning and monitoring can help mitigate scope creep and adapt Scrum practices to suit the specific needs of the project. By proactively addressing these potential challenges, teams can harness the power of Scrum to deliver high-quality results efficiently and effectively.'